In [1]:
words = open('names.txt', 'r').read().splitlines()

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
len(words)

32033

In [4]:
min(len(w) for w in words)

2

In [5]:
max(len(w) for w in words)

15

# Generate the bigrams

In [9]:
b = {}
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1

In [15]:
b.items()

dict_items([(('.', 'e'), 1531), (('e', 'm'), 769), (('m', 'm'), 168), (('m', 'a'), 2590), (('a', '.'), 6640), (('.', 'o'), 394), (('o', 'l'), 619), (('l', 'i'), 2480), (('i', 'v'), 269), (('v', 'i'), 911), (('i', 'a'), 2445), (('.', 'a'), 4410), (('a', 'v'), 834), (('v', 'a'), 642), (('.', 'i'), 591), (('i', 's'), 1316), (('s', 'a'), 1201), (('a', 'b'), 541), (('b', 'e'), 655), (('e', 'l'), 3248), (('l', 'l'), 1345), (('l', 'a'), 2623), (('.', 's'), 2055), (('s', 'o'), 531), (('o', 'p'), 95), (('p', 'h'), 204), (('h', 'i'), 729), (('.', 'c'), 1542), (('c', 'h'), 664), (('h', 'a'), 2244), (('a', 'r'), 3264), (('r', 'l'), 413), (('l', 'o'), 692), (('o', 't'), 118), (('t', 't'), 374), (('t', 'e'), 716), (('e', '.'), 3983), (('.', 'm'), 2538), (('m', 'i'), 1256), (('a', 'm'), 1634), (('m', 'e'), 818), (('.', 'h'), 874), (('r', 'p'), 14), (('p', 'e'), 197), (('e', 'r'), 1958), (('r', '.'), 1377), (('e', 'v'), 463), (('v', 'e'), 568), (('l', 'y'), 1588), (('y', 'n'), 1826), (('n', '.'), 6763

In [10]:
sorted(b.items(), key = lambda kv: -kv[1])

[(('n', '.'), 6763),
 (('a', '.'), 6640),
 (('a', 'n'), 5438),
 (('.', 'a'), 4410),
 (('e', '.'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('.', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('.', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '.'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('.', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', '.'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('.', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', '.'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('.', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('.', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('.', 'l'), 1572),
 (('.', 'c'), 1542),
 (('.', 'e'), 1531),
 (('j', 'a'), 1473),
 (('r', '.'), 1377),
 (('n', 'e'),

In [ ]:
t = {}
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    trigram = (ch1, ch2, ch3)
    t[trigram] = t.get(trigram, 0) + 1

In [ ]:
sorted(t.items(), key = lambda kv: -kv[1])

In [45]:
import torch
import torch.nn.functional as F

In [606]:
# --------- !!! OPTIMIZATION !!! yay, but this time actually --------------

In [89]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append(ix1 * 27 + ix2)
    ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

number of examples:  196113


In [86]:
xs[:10], ys[:10]

(tensor([  5, 148, 364, 352,  15, 417, 333, 265, 603, 244]),
 tensor([13, 13,  1,  0, 12,  9, 22,  9,  1,  0]))

In [87]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27 * 27, 27), generator=g, requires_grad=True)

In [90]:
# gradient descent
for k in range(200):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27*27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.001*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -60 * W.grad

2.84078049659729
2.8224966526031494
2.805215835571289
2.7888622283935547
2.773364782333374
2.7586593627929688
2.7446858882904053
2.731388568878174
2.718716621398926
2.706622838973999
2.6950647830963135
2.684002637863159
2.6733999252319336
2.6632251739501953
2.6534485816955566
2.644042491912842
2.634983539581299
2.6262495517730713
2.6178200244903564
2.609677314758301
2.60180401802063
2.5941851139068604
2.5868067741394043
2.5796561241149902
2.572721242904663
2.565990924835205
2.559455633163452
2.553104877471924
2.546931028366089
2.5409257411956787
2.535080671310425
2.5293900966644287
2.523846387863159
2.5184438228607178
2.513176441192627
2.5080385208129883
2.503025770187378
2.4981327056884766
2.4933550357818604
2.4886887073516846
2.484128713607788
2.4796719551086426
2.4753148555755615
2.4710540771484375
2.466885566711426
2.4628069400787354
2.458815336227417
2.454906940460205
2.451079845428467
2.447331428527832
2.4436588287353516
2.4400603771209717
2.436532735824585
2.433075189590454
2.42

In [91]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
  
  out = []
  last_ix = 0
  ix = 1
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([last_ix * 27 + ix]), num_classes=27*27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    #print(p)
    last_ix = ix
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    #print(ix)
    out.append(itos[ix])
    #fixme
    #break
    if ix == 0:
      break
    
  print(''.join(out))

dexzdfzjglkuriana.
vayhhmvlzimjtna.
n.
llayk.
ka.
da.
rleighuhptthrighton.
miliellavojkwpteda.
kaley.
m.
